In [89]:
! pip install -qU langchain openai

In [90]:
pip install langchain openai

Note: you may need to restart the kernel to use updated packages.


In [91]:
from langchain import PromptTemplate
import re
import tenacity
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

In [92]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [93]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string

In [94]:
character_names = ["Monica Geller", "Rachel Green", "Ross Geller"]
topic = "Go to Hospital to visit Phoebe"
word_limit = 3

In [95]:
from getpass import getpass
OPENAI_API_KEY=getpass("openAI API keys:")

openAI API keys:········


In [105]:
game_description = f"""Here is the topic for the thanksgiving dinner preparation: {topic}.
The persons involved are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each person involved."
)


def generate_character_description(character_name):
    if character_name == 'Rachel Green':
        character_description = f"""Rachel Green is a fashionable and initially spoiled young woman who ran away from her wedding to seek independence. At the beginning, she worked at Central Perk which is a coffeehouse.  She later pursues a successful career in the fashion industry. Rachel is caring and loyal to all her friends.
Rachel’s roommate is Monica who is her best friend.
Rachel knew Ross had a crush on her in high school, but she didn’t turn to that feeling until recently.
Rachel does not know Chandler well but they had already met several years ago at Monica’s parent’s house.
Rachel did not know Phoebe before she moved in with Monica, but they quickly became close friends. Rachel thinks Phoebe’s quirkiness is adorable despite that other people often make fun of it.
        """
    elif character_name == "Monica Geller":
        character_description = f"""Monica Geller works as a head chef. She was overweight when she was in high school and it affected her very much. It is part of the reason for her being a perfectionist, especially when it comes to cleanliness and order. She is emotionally strong and she always wants to get the best results even if it means through force. She loves being the hostess and loves taking care of her friends.
Monica is in a romantic relationship with Chandler now.
Monica lived with Phoebe before Rachel moved in. They remained best friends after Phoebe moved out.
Monica is Ross’s sister.
Monica has been friends with Rachel since they were younger.
Monica knows Joey and they soon become friends.
        """
    elif character_name == "Ross Geller":
        character_description = f"""Ross Geller is a paleontologist. As a scientist and professor, he thrives on accuracy, precision and being precise. He is a curious person, and is confident about his academic success.
Ross is Monica’s older brother.
Ross has an enduring love for Rachel.He had a crush on her when they met in high school.
Ross recently argued with Phoebe about evolution theory, and he goes to extreme lengths to prove his way is right.
Ross met Chandler in college and they remained best friends after college ended. Recently he finds out Monica and Chandler are in a relationship,and he is very supportive of their relationship.
Ross knows Joey through Chandler and they become good friends.
        """
    return character_description
'''
def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a creative description of the individual, {character_name}, in {word_limit} words or less, that emphasizes their personalities.
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
    character_description = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=1.0)(
        character_specifier_prompt
    ).content
    return character_description
'''

def generate_character_header(character_name, character_description):
    return f"""{game_description}
Your name is {character_name}.
You are one of the persons that is going to attend the thanksgiving diner party.
Your description is as follows: {character_description}
The topic you are discussing with others is: {topic}.
Your goal is to be as creative as possible, and contributes to the preparation about this topic,
"""


def generate_character_system_message(character_name, character_header):
    return SystemMessage(
        content=(
            f"""{character_header}
You will speak in the style of {character_name}, and exaggerate their personality.
You will come up with creative ideas related to {topic}.
Do not say the same things over and over again.
Speak in the first person from the perspective of {character_name}
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Stop speaking the moment you finish speaking from your perspective.
KEEP YOUR RESPONSES BRIEF!
Never forget to keep your response within {word_limit} sentences!
Keep your response within two paragraphs.
Do not add anything else.
    """
        )
    )


character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]
character_system_messages = [
    generate_character_system_message(character_name, character_headers)
    for character_name, character_headers in zip(character_names, character_headers)
]


'''
for (
    character_name,
    character_description,
    character_header,
    character_system_message,
) in zip(
    character_names,
    character_descriptions,
    character_headers,
    character_system_messages,
):
    print(f"\n\n{character_name} Description:")
    print(f"\n{character_description}")
    print(f"\n{character_header}")
    print(f"\n{character_system_message.content}")
'''

'\nfor (\n    character_name,\n    character_description,\n    character_header,\n    character_system_message,\n) in zip(\n    character_names,\n    character_descriptions,\n    character_headers,\n    character_system_messages,\n):\n    print(f"\n\n{character_name} Description:")\n    print(f"\n{character_description}")\n    print(f"\n{character_header}")\n    print(f"\n{character_system_message.content}")\n'

In [106]:
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets, like this: <int>."


bid_parser = BidOutputParser(
    regex=r"<(\d+)>", output_keys=["bid"], default_output_key="bid"
)

In [107]:
def generate_character_bidding_template(character_header):
    bidding_template = f"""{character_header}


{{message_history}}
On the scale of 1 to 10, where 1 is tasks you think you are really poor at and 10 is tasks you are supper good at,
rate the likelyhood of you managing to do the tasks in the following message
{{recent_message}}


{bid_parser.get_format_instructions()}
Do nothing else.
    """
    return bidding_template


'''
character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

for character_name, bidding_template in zip(
    character_names, character_bidding_templates
):
    print(f"{character_name} Bidding Template:")
    print(bidding_template)
'''

'\ncharacter_bidding_templates = [\n    generate_character_bidding_template(character_header)\n    for character_header in character_headers\n]\n\nfor character_name, bidding_template in zip(\n    character_names, character_bidding_templates\n):\n    print(f"{character_name} Bidding Template:")\n    print(bidding_template)\n'

In [108]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}

        You are the Discussion moderator.
        Please make the dinner preparation topic more specific.
        Frame the dinner preparation topic as a problem to be solved.
        Be creative and imaginative.
        Please reply with the specified topic in {word_limit} words or less.
        Do keep your responses brief!
        Speak directly to the persons who are initiate the party: {*character_names,}.
        Do not add anything else."""
    ),
]

specified_topic = topic
# specified_topic = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=1.0)(topic_specifier_prompt).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
Go to Hospital to visit Phoebe

Detailed topic:
Go to Hospital to visit Phoebe



Speaker selection function

In [109]:
@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),
    wait=tenacity.wait_none(),  # No waiting time between retries
    retry=tenacity.retry_if_exception_type(ValueError),
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),
    retry_error_callback=lambda retry_state: 0,
)  # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)["bid"])
    return bid



In [110]:
import numpy as np


def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    print("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"Selected: {selected_name}")
    print("\n")
    return idx

Main Loop

In [111]:
characters = []
for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.2),
            bidding_template=bidding_template,
        )
    )

In [86]:
!pip install gradio

In [112]:
max_iters = 15
n = 0

simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Discussion Moderator", specified_topic)
print(f"(Discussion Moderator): {specified_topic}")
print("\n")
'''
while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1
'''
import gradio as gr

def get_new_message(chat_log):
    name, message = simulator.step()

    new_entry = f"{(name)}: {message}\n\n-----------------------------------------------------------------------\n"
    updated_chat_log = chat_log + new_entry
    return updated_chat_log


with gr.Blocks() as demo:
    chat_log = gr.Textbox(interactive=False, lines=30, label=topic, value="")

    update_button = gr.Button("Get New Message")

    update_button.click(
        get_new_message,
        inputs=[chat_log],
        outputs=[chat_log]
    )

demo.launch(share=True)


(Discussion Moderator): Go to Hospital to visit Phoebe


Running on local URL:  http://127.0.0.1:7873
Running on public URL: https://06f1f43a9b2c1ea0d6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Bids:
	Monica Geller bid: 9
	Rachel Green bid: 8
	Ross Geller bid: 9
Selected: Monica Geller


Bids:
	Monica Geller bid: 10
	Rachel Green bid: 9
	Ross Geller bid: 8
Selected: Monica Geller


Bids:
	Monica Geller bid: 9
	Rachel Green bid: 9
	Ross Geller bid: 9
Selected: Rachel Green


Bids:
	Monica Geller bid: 10
	Rachel Green bid: 8
	Ross Geller bid: 8
Selected: Monica Geller


Bids:
	Monica Geller bid: 9
	Rachel Green bid: 8
	Ross Geller bid: 9
Selected: Ross Geller


